In [1]:
#!/usr/bin/env python
# Copyright 2021-2024 The PySCF Developers. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import pyscf
from gpu4pyscf.scf import hf
from gpu4pyscf.qmmm.pbc import itrf, mm_mole

import numpy as np

atom ='''
O       0.0000000000    -0.0000000000     0.1174000000
H      -0.7570000000    -0.0000000000    -0.4696000000
H       0.7570000000     0.0000000000    -0.4696000000
'''

mol = pyscf.M(atom=atom, basis='3-21g', max_memory=40000)

mol.verbose = 4
mf_GPU = hf.RHF(mol)

# Add mm charges:
# -0.8 charge at 1.0, 2.0,-1.0
#  0.8 charge at 3.0, 4.0, 5.0
# Place the QM and MM atoms in a box of 12 A
# Real-space cutoff for Ewald set to 8 A
# Exact MM potential computed for MM charges within 6 A of QM geometric center
mf_GPU = itrf.add_mm_charges(mf_GPU, [[1,2,-1],[3,4,5]], np.eye(3)*12, [-0.8,0.8], [0.8,1.2], rcut_ewald=8, rcut_hcore=6)

# Compute Energy of QM-QM and QM-MM but NOT MM-MM
e_dft = mf_GPU.kernel()
print(f"total energy = {e_dft}")

# Compute Gradient
g = mf_GPU.nuc_grad_method()
g.max_memory = 40000
g.auxbasis_response = True
# energy gradient w.r.t. QM atom positions
g_dft = g.kernel()
# energy gradient w.r.t. MM atom positions
g_mm = g.grad_nuc_mm() + g.grad_hcore_mm(mf_GPU.make_rdm1()) + g.de_ewald_mm



******** <class 'gpu4pyscf.qmmm.pbc.itrf.QMMMRHF'> ********
method = QMMMRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'gpu4pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
max_memory 40000 MB (current use 497 MB)
** Add background charges for QMMMRHF **
lattice vectors  a1 [22.676713495, 0.000000000, 0.000000000]
                 a2 [0.000000000, 22.676713495, 0.000000000]
                 a3 [0.000000000, 0.000000000, 22.676713495]
Set gradient conv threshold to 3.16228e-05
1 MM charges see directly QM density
init E= -75.4172905365797
cycle= 1 E= -75.5573455047398  delta_E= -0.14  |ddm|= 1.22
cycle= 2 E= -75.5896385429497  delta_E= -0.0323  |ddm|= 0.373
cycle= 3 E= -75.59656825537  delta_E= -0.00693  |ddm|= 0.105
cycle= 4 E= -75.5967434805786  delta_E= -0.000175  |ddm|= 0.0238
cycle= 5 E= -75.5967496273813  delta_E= 

<class 'pyscf.lib.misc.QMMMGradients'> does not have attributes  auxbasis_response


--------------- QMMMRHF gradients ---------------
         x                y                z
0 O     0.0038784972     0.0224561095    -0.0112524641
1 H     0.0081399860    -0.0087740361     0.0037014969
2 H    -0.0113256726    -0.0231919330     0.0024562407
----------------------------------------------


In [9]:
mf_GPU.mol.atom_coords(), mf_GPU.mol.atom_mass_list(), mf_GPU.mol.atom_charges()

(array([[ 0.        ,  0.        ,  0.22185385],
        [-1.43052268,  0.        , -0.88741539],
        [ 1.43052268,  0.        , -0.88741539]]),
 array([16,  1,  1]),
 array([8, 1, 1], dtype=int32))

In [11]:
mf_GPU.mm_mol.atom_coords(), mf_GPU.mm_mol.atom_charges()

(array([[ 1.88972612,  3.77945225, -1.88972612],
        [ 5.66917837,  7.5589045 ,  9.44863062]]),
 array([-0.8,  0.8]))